In [1]:
import pandas as pd
import numpy as np
from datetime import date
import time
from datetime import datetime, timedelta
import os
import string

import httplib2
from googleapiclient import discovery
from oauth2client.service_account import ServiceAccountCredentials
import urllib
import requests
import codecs
import json
from pandas import json_normalize
from tqdm.notebook import tqdm

#Аутентификация для Google docs
CREDENTIALS_FILE = 'bsd-credentials.json'  # имя файла с закрытым ключом
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_FILE,\
                            ['https://www.googleapis.com/auth/spreadsheets',\
                             'https://www.googleapis.com/auth/drive'])
service = discovery.build('sheets', 'v4', credentials=credentials)
spreadsheetId = '1-KJwJoBHr4XDSFkkhuoT3ZS0VPQzfk_VdpGqjrl8398'

#Аутентификация для Youscan
AUTH_TOKEN = "b1b0c10a222647c8"


## Сохранение данных о загруженных из Youscan датасетах в GoogleDocs

In [2]:
#необходимый элемент интеграции с эксель
alph = list(string.ascii_uppercase) #буквы от A до Z
#В надежде, что больше колонок не будет. Если будет, то потом добавлю :)
excel_columns = dict(zip(list(range(1, len(alph)+1)), alph))

In [15]:
#Подключаемся к Youscan
req = requests.get("https://api.youscan.io/api/external/topics/", params={
        'apiKey': AUTH_TOKEN
    })

#Берем из него все названия тем.
bsd_names = pd.DataFrame(req.json()['topics'])
#Все темы, касающиеся BSD обозваны через суффикс ' - BSD'
bsd_names = bsd_names[bsd_names['name'].str.contains(' - BSD')]

In [16]:
#Проверяем
bsd_names

,id,name,query,filter,ocrQuery
0,254193,Agusha - BSD,Агуша или (фруто /1 няня) или фрутоняня или hi...,"{'country': {'include': ['ru'], 'missing': 'in...",NaN
2,253983,MMX - BSD,Эргоферон или (Анаферон или (Анаферон и Детски...,"{'country': {'include': ['ru'], 'missing': 'in...",NaN
3,254506,S7 - BSD,"((сибирские /1 авиалинии) или ""s7airlines"" или...","{'country': {'include': ['ru'], 'missing': 'in...",NaN


Теперь для каждой тема смотрим, есть ли уже выгрузка на сервере. Если есть, до докачиваем данные от последнего дня до сегодня.
Если нет, то качаем все, что есть. Сохраняем выгруженное. После этого заносим в гугл-таблицу длину нового файла в строку с номером текущего месяца и в столбец с названием темы.

In [5]:
def replace_semicolon(x):
    if isinstance(x, str):
        return x.replace(';', ",")
    else:
        return x

In [18]:
#Проходимся по строчкам таблицы с существующими сейчас в Youscan темами
for i in [3]:
    name = bsd_names.loc[i, :]['name'] #имя темы
    name = name.split()[0]
    topic_id = bsd_names.loc[i, :]['id'] #topic_id для обращения к Api
    print(name)
    
    #Ищем существующие файлы для этой темы на сервере
    today = (datetime.now() + timedelta(hours=3)).date().strftime('%Y-%m-%d')
    files = os.listdir('../DATA/BrandSocmediaDashboard')
    df_file = [f for f in files if (name+'_total').lower() in f.lower()]
    #Если таковых не обнаружено
    if len(df_file)<1:
        #Тогда считаем, что первая дата - это два года назад
        #На большее все равно надо запрашивать шлубинный сбор
        #И если будет так, это скорее всего будет обрабатываться отдельно
        last_day = str(int(today.split('-')[0])-2)+'-'+today[-5:]
        total_df = pd.DataFrame()
        print('Первая выгрузка')
    #если таковые есть
    else:
        df_file = df_file[0]
        total_df = pd.read_csv('../DATA/BrandSocmediaDashboard/'+df_file, sep=';')
        total_df = total_df[total_df['fullText'].notna()&total_df['published'].notna()]
        total_df = total_df[total_df['published'].str.contains('T')]
        last_day = str(total_df['published'].max()).split('T')[0]
        print('Дата последней выгрузки: ', last_day)
   
    #Проверим, не выгружали ли мы сегодня эти же данные
    #И выкачаем их из Youscan 
    #Схема обращения стандартная
    if (last_day!=today):
        req_num = requests.get("https://api.youscan.io/api/external/topics/%s/mentions/" % (topic_id), params={
                'apiKey': AUTH_TOKEN,
                'from': f'{last_day}T00:00',
                'to': f'{today}T00:00',      
                'size': 1,
                'fullText': True,
                'orderBy' : "seqAsc",
                'sinceSeq': 0
            })
        df = pd.DataFrame()
        firstSeq = min([m['seq'] for m in req_num.json()['mentions']])
        lastSeq = req_num.json()['lastSeq']
        total_mentions = lastSeq - firstSeq
        sinceSeq = 0
        
        for i in tqdm(range(int(total_mentions//1000+1))): # т.к. выгружать можем олько по 1к то проходимся по всему
    
            sinceSeq = i*1000
            req = requests.get("https://api.youscan.io/api/external/topics/%s/mentions/" % (topic_id), params={
                    'apiKey': AUTH_TOKEN,
                    'from': f'{last_day}T00:00',
                    'to': f'{today}T00:00',             
                    'size': 1000,
                    'fullText': True,
                    'orderBy' : "seqAsc",
                    'sinceSeq': firstSeq + sinceSeq 
                    })
            try:
                df = df.append(json_normalize(req.json()['mentions'])) # добавляем в df каждую новую тысяч
            except:
                pass
        df = df.drop_duplicates(subset=['postId'])

        df.to_csv(f'../DATA/BrandSocmediaDashboard/youscan_{name.lower()}_{today}.csv', index=False)
        total_df = pd.concat([total_df, df], sort=True)
        total_df = total_df.drop_duplicates(subset=['postId'])
        total_df = total_df.applymap(replace_semicolon)
        total_df.to_csv(f'../DATA/BrandSocmediaDashboard/youscan_{name.lower()}_total.csv', sep=';', index=False)
        
    #Теперь надо отчитаться в Google Docs        
    result = service.spreadsheets().values().get(
        spreadsheetId=spreadsheetId, range='plan').execute()
    #Посмотрим на то, как выглядит наша таблица
    rows = result.get('values', [])
    gd_df = pd.DataFrame(rows)
    gd_df = gd_df.set_index(0, drop=True)
    gd_df.columns = gd_df.iloc[0, :]
    gd_df = gd_df[1:].fillna(0)
    gd_df = gd_df.replace({'': 0})
    
    #Посмотрим, записанно ли какое-нибудь число в ячейке, соответсвующей текущим месяцу и теме
    num = gd_df[name][today[:-3]]
    #Если там не ноль
    if (num==0)|(num=='0'):
        # +2 - это потому что нумерация начинается с 1, а еще потому что первая позиция - это индекс
        row = int(np.where(gd_df.index==today[:-3])[0])+2
#         row = int(np.where(gd_df.index=='2021-03')[0])+2
        column = excel_columns[int(np.where(gd_df.columns==name)[0])+2]
        #Указание колонок согласно api google docs
        range_ = f'plan!{column}{row}:{column}{row}' 
        #Чтобы умная машина не пыталась решать за тебя, как ты хочешь заносить данные
        value_input_option = 'RAW'
#         insert_data_option = 'OVERWRITE' 
        value_range_body = {
              "majorDimension": "ROWS",
              "values": [[len(total_df)]] #потому что записываешь ты кусок таблицы - строки и столбцы
    }

    #А теперь скормим все это в api
        request = service.spreadsheets().values().update(spreadsheetId=spreadsheetId, range=range_, valueInputOption=value_input_option, body=value_range_body)
        response = request.execute()
        
        
        
    print('---------')


S7


/home/analyst/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (5,13,14,15,16,18,19,20,21,22,23,44) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Дата последней выгрузки:  2021-03-31


KeyboardInterrupt: 

In [36]:
#Посмотрим, записанно ли какое-нибудь число в ячейке, соответсвующей текущим месяцу и теме
num = gd_df[name][today[:-3]]
#Если там не ноль
if (num==0)|(num=='0'):
    # +2 - это потому что нумерация начинается с 1, а еще потому что первая позиция - это индекс
    row = int(np.where(gd_df.index==today[:-3])[0])+2
    column = excel_columns[int(np.where(gd_df.columns==name)[0])+2]
    #Указание колонок согласно api google docs
    range_ = f'plan!{column}{row}:{column}{row}' 
    #Чтобы умная машина не пыталась решать за тебя, как ты хочешь заносить данные
    value_input_option = 'RAW'
#         insert_data_option = 'OVERWRITE' 
    value_range_body = {
          "majorDimension": "ROWS",
          "values": [[len(total_df)]] #потому что записываешь ты кусок таблицы - строки и столбцы
}

#А теперь скормим все это в api
    request = service.spreadsheets().values().update(spreadsheetId=spreadsheetId, range=range_, valueInputOption=value_input_option, body=value_range_body)
    response = request.execute()

In [21]:
name[:-6]

'MMX'

In [26]:
result = service.spreadsheets().values().get(
    spreadsheetId=spreadsheetId, range='plan').execute()
#Посмотрим на то, как выглядит наша таблица
rows = result.get('values', [])
df = pd.DataFrame(rows)

In [28]:
df.replace({'': 0})

,0,1,2,3,4,5,6,7,8
0,0,MMX,Agusha,Adrenaline,LADA,Retail,Henkel,Alfabank,Daimler
1,2020-12,703440,324002,240223,564111,521506,0,94912,0
2,2021-01,0,0,256162,0,855657,0,100347,None
3,2021-02,None,None,None,None,None,None,None,None
4,2021-03,None,None,None,None,None,None,None,None
